<h2 style="color:crimson">Ejercicios</h2>

Considerando que la relación dentro de la base de datos `nabla_python` tiene la siguiente estructura:

![relation](http://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)


1. Realiza una consulta a la base de datos `nabla_python` por medio de `pymysql`: encuentra los tipos de `status` existentes dentro de la tabla `orders`.

---

2. Realiza una consulta a la base de datos `nabla_python` por medio de `pymysql`: encuentra los 10 clientes `customerName` que más han comprado en la tienda. La tabla resultante deberá estar ordenada de mayor a menor con `customerName` y `expenses` como la columna en dónde se encuentre el monto gastado en la tienda. (**hint**: considera las tablas `customers` y `payments`)
----

3. Escribe una consulta que, para cada producto comprado muestre la clave del cliente `customerNumber` y el código del producto `productCode`. Obten los primeros 10 valores

---

4. En base al ejercicio anterior, reemplaza `productCode` por `productName`. **HINT**: El comando `WITH` nos ayuda a crear una tabla temporal:

```SQL
WITH t1 as (
    SELECT ...
    )
SELECT t1.ci, t2.cj
FROM t2
JOIN t1 ON t1.cx = t2.cx
```

---

5. En base al ejercicio anterior, reemplaza `customerNumber` por `customerName`

----

6. En base al ejercicio 3, encuentra el nombre del producto `productName` y cuantas unidades se han comprado de ese producto. Ordena los elementos de mayor a menor.

---

7. Por medio de SQLAlchemy, define la tabla `orders` definida dentro de `nabla_python`

```
+----------------+-------------+------+-----+---------+-------+
| Field          | Type        | Null | Key | Default | Extra |
+----------------+-------------+------+-----+---------+-------+
| orderNumber    | int(11)     | NO   | PRI | NULL    |       |
| orderDate      | date        | NO   |     | NULL    |       |
| requiredDate   | date        | NO   |     | NULL    |       |
| shippedDate    | date        | YES  |     | NULL    |       |
| status         | varchar(15) | NO   |     | NULL    |       |
| comments       | text        | YES  |     | NULL    |       |
| customerNumber | int(11)     | NO   | MUL | NULL    |       |
+----------------+-------------+------+-----+---------+-------+
```

----

8. Por medio de SQLAlchemy, define la tabla `orderdetails` definida dentro de `nabla_python`

```
+-----------------+---------------+------+-----+---------+-------+
| Field           | Type          | Null | Key | Default | Extra |
+-----------------+---------------+------+-----+---------+-------+
| orderNumber     | int(11)       | NO   | PRI | NULL    |       |
| productCode     | varchar(15)   | NO   | PRI | NULL    |       |
| quantityOrdered | int(11)       | NO   |     | NULL    |       |
| priceEach       | decimal(10,2) | NO   |     | NULL    |       |
| orderLineNumber | smallint(6)   | NO   |     | NULL    |       |
+-----------------+---------------+------+-----+---------+-------+
```

---

9. Realiza una consulta a la base de datos `nabla_python` por medio de `sqlalchemy`: encuentra los tipos de `status` existentes dentro de la tabla `orders`.

In [1]:
import pymysql
passw = "maquina271"
conn = pymysql.connect(host="db4free.net",
                       user="nabla123",
                       port=3306,
                       password=passw,
                       database="nabla_python",
                       charset="utf8")

1. Realiza una consulta a la base de datos nabla_python por medio de pymysql: encuentra los tipos de status existentes dentro de la tabla orders.

In [5]:
query = "SELECT DISTINCT status FROM orders;"
with conn.cursor() as cursor:
    cursor.execute(query)
    orders = cursor.fetchmany(5)
    
for order in orders:
    print(order)

('Shipped',)
('Resolved',)
('Cancelled',)
('On Hold',)
('Disputed',)


2. Realiza una consulta a la base de datos nabla_python por medio de pymysql: encuentra los 10 clientes customerName que más han comprado en la tienda. La tabla resultante deberá estar ordenada de mayor a menor con customerName y expenses como la columna en dónde se encuentre el monto gastado en la tienda. (hint: considera las tablas customers y payments)

In [86]:
query = """
SELECT customerName, sum(amount) as expenses
FROM customers
JOIN payments
ON payments.customerNumber = customers.customerNumber
GROUP BY customerName
ORDER BY expenses desc;
"""

with conn.cursor() as cursor:
    cursor.execute(query)
    elements = cursor.fetchmany(10)
idx = 0     
for element in elements:
    idx += 1 
    print(f'{idx:<2} {element[0]:<30}:  {element[1]:,.2f}')
   
   

1  Euro+ Shopping Channel        :  715,738.98
2  Mini Gifts Distributors Ltd.  :  584,188.24
3  Australian Collectors, Co.    :  180,585.07
4  Muscle Machine Inc            :  177,913.95
5  Dragon Souveniers, Ltd.       :  156,251.03
6  Down Under Souveniers, Inc    :  154,622.08
7  AV Stores, Co.                :  148,410.09
8  Anna's Decorations, Ltd       :  137,034.22
9  Corporate Gift Ideas Co.      :  132,340.78
10 Saveley & Henriot, Co.        :  130,305.35


3. Escribe una consulta que, para cada producto comprado muestre la clave del cliente customerNumber y el código del producto productCode. Obten los primeros 10 valores

In [100]:
#Asumiendo que comprado significa que la orden esta "resolved" o "shipped"
# no sé si requería otro campo, supongo que no.

query = """SELECT  A.customerNumber, B.productCode

        FROM orders as A
        JOIN orderdetails as B
        ON B.orderNumber = A.orderNumber
        
        WHERE A.status in ('Shipped','Resolved')
        GROUP BY A.customerNumber, B.productCode
        ;"""
with conn.cursor() as cursor:
    cursor.execute(query)
    orders = cursor.fetchmany(10)
    
for order in orders:
    print(order)

(363, 'S18_1749')
(363, 'S18_2248')
(363, 'S18_4409')
(363, 'S24_3969')
(128, 'S18_2325')
(128, 'S18_2795')
(128, 'S24_1937')
(128, 'S24_2022')
(181, 'S18_1342')
(181, 'S18_1367')


4. En base al ejercicio anterior, reemplaza productCode por productName. HINT: El comando WITH nos ayuda a crear una tabla temporal:

In [102]:
query = """WITH D as (SELECT  B.orderNumber,B.productCode, C.productName
                        FROM orderdetails as B
                        JOIN products as C
                        ON B.productCode = C.productCode)
        
        SELECT  A.customerNumber, D.productName

        FROM orders as A
        JOIN D
        ON D.orderNumber = A.orderNumber
        
        WHERE A.status in ('Shipped','Resolved')
        GROUP BY A.customerNumber, D.productName
        ;"""
with conn.cursor() as cursor:
    cursor.execute(query)
    orders = cursor.fetchmany(10)
    
for order in orders:
    print(order)

(363, '1917 Grand Touring Sedan')
(363, '1911 Ford Town Car')
(363, '1932 Alfa Romeo 8C2300 Spider Sport')
(363, '1936 Mercedes Benz 500k Roadster')
(128, '1932 Model A Ford J-Coupe')
(128, '1928 Mercedes-Benz SSK')
(128, '1939 Chevrolet Deluxe Coupe')
(128, '1938 Cadillac V-16 Presidential Limousine')
(181, '1937 Lincoln Berline')
(181, '1936 Mercedes-Benz 500K Special Roadster')


5. En base al ejercicio anterior, reemplaza customerNumber por customerName

In [106]:
query = """WITH D as (SELECT  B.orderNumber,B.productCode, C.productName
                        FROM orderdetails as B
                        JOIN products as C
                        ON B.productCode = C.productCode)
            
        SELECT  E.customerName, D.productName

        FROM orders as A
        JOIN D
        ON D.orderNumber = A.orderNumber
        JOIN customers as E
        ON E.customerNumber = A.customerNumber
        WHERE A.status in ('Shipped','Resolved')
        GROUP BY A.customerNumber, D.productName
        ;"""
with conn.cursor() as cursor:
    cursor.execute(query)
    orders = cursor.fetchmany(10)
    
for order in orders:
    print(order)

('Online Diecast Creations Co.', '1917 Grand Touring Sedan')
('Online Diecast Creations Co.', '1911 Ford Town Car')
('Online Diecast Creations Co.', '1932 Alfa Romeo 8C2300 Spider Sport')
('Online Diecast Creations Co.', '1936 Mercedes Benz 500k Roadster')
('Blauer See Auto, Co.', '1932 Model A Ford J-Coupe')
('Blauer See Auto, Co.', '1928 Mercedes-Benz SSK')
('Blauer See Auto, Co.', '1939 Chevrolet Deluxe Coupe')
('Blauer See Auto, Co.', '1938 Cadillac V-16 Presidential Limousine')
('Vitachrome Inc.', '1937 Lincoln Berline')
('Vitachrome Inc.', '1936 Mercedes-Benz 500K Special Roadster')
